In [ ]:
!pip install -q pandas chromadb sentence-transformers

In [ ]:
%pip install -q pandas chromadb sentence-transformers

In [ ]:
import os
import mykeys


In [ ]:
mykeys.setOsEnv()

In [50]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import openai
import json
import os
import time
import concurrent.futures
import glob
import ast # (★★★ 1단계 NameError 방지용 ★★★)
import chromadb # (★★★ 2단계 NameError 방지용 ★★★)
import chromadb.utils.embedding_functions as embedding_functions # (★★★ 2단계 NameError 방지용 ★★★)
import random # (★★★ "랜덤 평가"를 위해 추가 ★★★)

In [54]:
# --- 1. 설정 ---
RESTAURANT_DB_FILE = "restaurant_summaries_output_ALL.csv"
USER_PROFILE_FILE = "user_profiles_for_hybrid_search.csv"
DB_PERSISTENT_PATH = "./restaurant_db"
COLLECTION_NAME = "restaurants"

# (★★★ 신규 ★★★)
# 최종 추천 결과를 저장할 파일
RECOMMENDATION_OUTPUT_FILE = "recommendation_results_with_ratings.csv"

# (★★★ 핵심 설정 ★★★)
# DB를 강제로 새로 만들고 싶을 때만 True로 설정
CLEAR_DB_AND_REBUILD = False

# 테스트할 사용자 프로필 수. (0 또는 500으로 설정 시 전체 실행)
NUM_PROFILES_TO_TEST = 0


In [42]:

# (★★★ LLM 쿼리 재작성을 위한 설정 ★★★)
GPT_API_NAME = "gpt-4.1-mini" 
try:
  client = openai.OpenAI()
  if not client.api_key:
    raise openai.OpenAIError("OPENAI_API_KEY 환경 변수가 설정되지 않았습니다.")
except Exception as e:
  print(f"API 키 로드 오류: {e}")
  exit()

In [43]:
# --- 2. 헬퍼 함수 (1/7): 레스토랑 CSV 로더 ---
def load_and_prepare_data(csv_path):
  """
  restaurant_summaries_output...csv 파일을 로드하고
  '메타데이터' 컬럼을 딕셔너리로 변환합니다.
  """
  print(f"'{csv_path}' 파일 로드 중...")
  try:
    df = pd.read_csv(csv_path)
  except FileNotFoundError:
    print(f"[오류] 파일을 찾을 수 없습니다: {csv_path}")
    return None

  def safe_convert_to_dict(x):
    if pd.isna(x) or x == '{}' or x == '':
      return {}
    try:
      return ast.literal_eval(x)
    except Exception as e:
      print(f"메타데이터 파싱 오류: {e} \n데이터: {x}")
      return {"error": "parsing_failed"}

  print("메타데이터 컬럼을 딕셔너리로 변환 중...")
  df['메타데이터'] = df['메타데이터'].apply(safe_convert_to_dict)
  df['RAG텍스트'] = df['RAG텍스트'].fillna('')
  
  print(f"데이터 준비 완료: {len(df)}개")
  return df

In [44]:
# --- 3. 헬퍼 함수 (2/7): 벡터 DB 구축 (Persistence 적용) ---
def build_vector_db(df, clear_db=False):
  """
  레스토랑 DataFrame을 받아 ChromaDB를 구축하거나 로드합니다.
  """
  print("\n--- 2단계: VectorDB 구축/로드 시작 ---")
  
  # 1. 임베딩 모델 준비
  model_name = "distiluse-base-multilingual-cased-v1"
  sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=model_name
  )
  
  # 2. ChromaDB 클라이언트 초기화 (파일 기반 Persistent)
  print(f"'{DB_PERSISTENT_PATH}' 경로에서 Persistent DB 클라이언트를 초기화합니다...")
  client = chromadb.PersistentClient(path=DB_PERSISTENT_PATH)

  # 3. CLEAR 옵션 처리
  if clear_db:
    try:
      print(f"[경고] CLEAR_DB_AND_REBUILD=True. '{COLLECTION_NAME}' 컬렉션을 삭제합니다.")
      client.delete_collection(name=COLLECTION_NAME)
    except Exception as e:
      print(f"  > 컬렉션 삭제 실패 (무시): {e}")
      pass

  # 4. DB 로드 시도
  try:
    print(f"기존 '{COLLECTION_NAME}' 컬렉션 로드를 시도합니다...")
    collection = client.get_collection(
      name=COLLECTION_NAME,
      embedding_function=sentence_transformer_ef
    )
    print(f"--- 2단계: VectorDB 로드 완료 ---")
    print(f"총 {collection.count()}개의 레스토랑이 (파일) DB에서 로드되었습니다.")
    return collection
  
  except Exception as e:
    print(f"  > 기존 컬렉션을 찾을 수 없습니다. (이유: {e})")
    print("  > 새 컬렉션을 생성하고 데이터 적재를 시작합니다. (시간 소요)")
    pass 

  # 5. DB 생성 및 적재 (로드 실패 시에만 실행)
  try:
    collection = client.create_collection(
      name=COLLECTION_NAME,
      embedding_function=sentence_transformer_ef
    )
  except Exception as e:
    print(f"[오류] 컬렉션 생성 실패: {e}")
    return None

  # 6. 데이터 DB에 적재 (데이터 전처리)
  documents_list = df['RAG텍스트'].tolist()
  metadatas_list = df['메타데이터'].tolist() 
  ids_list = df['id'].astype(str).tolist()

  print("ChromaDB 호환을 위해 metadata의 list 값을 string으로, None 값을 ''로 변환 중...")
  processed_metadatas = []
  for metadata_dict in metadatas_list:
    processed_meta_item = {}
    for key, value in metadata_dict.items():
      if value is None:
        processed_meta_item[key] = "" 
      elif isinstance(value, list):
        processed_meta_item[key] = ",".join(map(str, value))
      else:
        processed_meta_item[key] = value
    processed_metadatas.append(processed_meta_item)

  # 7. 배치(Batch) 처리
  print(f"ChromaDB에 데이터 {len(ids_list)}개 적재 중...")
  BATCH_SIZE = 5000 
  total_size = len(ids_list)
  
  for i in range(0, total_size, BATCH_SIZE):
    end_i = min(i + BATCH_SIZE, total_size)
    batch_docs = documents_list[i:end_i]
    batch_metas = processed_metadatas[i:end_i]
    batch_ids = ids_list[i:end_i]
    
    print(f"  > 배치 {i//BATCH_SIZE + 1}/{ -(-total_size // BATCH_SIZE)} ({i+1}~{end_i}번) 적재 중...")
    collection.add(
      documents=batch_docs,
      metadatas=batch_metas,
      ids=batch_ids
    )

  print("--- 2단계: VectorDB 신규 구축 및 적재 완료 ---")
  print(f"총 {collection.count()}개의 레스토랑이 (파일) DB에 저장되었습니다.")
  return collection

In [35]:
# --- 4. 헬퍼 함수 (3/7): 하이브리드 검색 필터 빌더 ---
def build_filters_from_profile(user_filter_dict):
  """
  사용자 프로필 딕셔너리를 받아 ChromaDB 1차 필터와 Python 2차 필터로 분리합니다.
  """
  db_pre_filter_list = [] 
  python_post_filter_dict = {} 
  
  DB_FILTER_KEYS = ['budget_range', 'spicy_available', 'vegetarian_options']
  POST_FILTER_KEYS = ['main_ingredients_list', 'suitable_for']

  for key, value in user_filter_dict.items():
    if value == 'N/A' or not value: 
      continue
      
    if key == 'food_category':
      # 사용자의 'food_category'는 가게 DB의 'high_level_category'와 매칭
      db_pre_filter_list.append({"high_level_category": value})
      
    elif key in DB_FILTER_KEYS:
      db_pre_filter_list.append({key: value})
      
    elif key in POST_FILTER_KEYS:
      python_post_filter_dict[key] = value.split(',')
      
  db_pre_filter = {"$and": db_pre_filter_list} if db_pre_filter_list else {}
  
  return db_pre_filter, python_post_filter_dict

In [45]:
# --- 4. 헬퍼 함수 (3/7): RAG 쿼리 재작성 ---
def generate_rag_query(user_profile_summary):
  """
  LLM을 호출하여 긴 자기소개(요약문)를
  가게 RAG 텍스트와 매칭하기 좋은 '짧은 핵심 쿼리'로 변환합니다.
  """
  print("  > [RAG] LLM을 호출하여 '분위기/성향' 쿼리를 재작성합니다...")
  
  system_prompt = """
  당신은 사용자의 긴 자기소개 텍스트를, 레스토랑 벡터 DB에서 검색하기 위한
  '짧고 핵심적인 쿼리 문장'으로 재작성(Re-writing)하는 전문가입니다.
  
  [규칙]
  1.  '안녕하세요', '저는 OOO입니다', '30대', '캐나다' 등 개인 신상 정보는 *모두 제거*합니다.
  2.  '예산(저/중/고)', '맵기(O/X)', '선호 재료(소고기)' 등 '사실(Fact)' 정보는 *모두 제거*합니다.
  3.  오직 사용자가 원하는 *분위기*, *상황*, *경험*, *성향* (예: '조용한', '혼자', '연인과 함께', '새로운 도전', '인기 맛집', '가족적인')만 추출하여 하나의 문장으로 만듭니다.
  4.  결과는 오직 '재작성된 쿼리 문장' 하나만 반환합니다.
  """
  
  user_prompt = f"""
  [사용자 자기소개]
  {user_profile_summary}
  
  [재작성된 쿼리]
  """

  try:
    response = client.chat.completions.create(
      model=GPT_API_NAME,
      messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
      ],
      temperature=0.2
    )
    rewritten_query = response.choices[0].message.content.strip().replace('"', '')
    return rewritten_query
  except Exception as e:
    print(f"  > [오류] 쿼리 재작성 실패: {e}")
    return user_profile_summary[:150]

In [46]:
# --- 5. 헬퍼 함수 (4/7): 하이브리드 검색 실행 (점수제) ---
def run_hybrid_search_with_scoring(collection, user_profile_row, df_restaurants, n_results=5):
  """
  (수정됨) RAG-First 검색 후, 메타데이터로 '점수'를 매겨 랭킹을 정합니다.
  """
  print("\n--- 3단계: RAG + 점수제(Scoring) 검색 시작 ---")
  
  # 1. 사용자 프로필(Pandas Row)에서 데이터 추출
  try:
    user_original_summary = user_profile_row['rag_query_text']
    user_filter_dict = ast.literal_eval(user_profile_row['filter_metadata_json'])
  except Exception as e:
    print(f"[오류] 사용자 프로필 파싱 실패: {e}")
    return [] # (★★★ 수정) 오류 시 빈 리스트 반환

  # 2. (핵심 수정) RAG 쿼리 재작성
  user_rag_query = generate_rag_query(user_original_summary)
  
  REQUEST_N_RESULTS = 50 # (★★★ RAG 검색으로 50개를 먼저 확보 ★★★)

  print(f"테스트 사용자: {user_profile_row['name']} ({user_profile_row['user_id']})")
  print(f"RAG 쿼리 (재작성됨): '{user_rag_query}'")
  print(f"Python 2차 필터 (점수 계산용): {user_filter_dict}")

  # 3. (★★★ 핵심 수정) ChromaDB에 *필터 없이* RAG 검색 실행
  try:
    print(f"\n--- 1단계: RAG 검색 (필터 없음) | Top {REQUEST_N_RESULTS}개 찾기 ---")
    results = collection.query(
      query_texts=[user_rag_query],
      n_results=REQUEST_N_RESULTS
      # (where= 필터 없음!)
    )
    
    print(f"--- 1차 검색 완료: {len(results['ids'][0])}개 후보 반환 ---")
    if not results.get('ids', [[]])[0]:
      print("RAG 검색 결과가 없습니다.")
      return []

    # 4. (★★★ 핵심 수정) Python으로 *점수(Scoring)* 계산
    final_results_with_score = []
    
    print("\n--- 2단계: 점수(Scoring) 계산 시작 (1차 후보 50개 대상) ---")
    
    for i in range(len(results['ids'][0])):
      store_id = results['ids'][0][i]
      rag_distance = results['distances'][0][i] # RAG 점수 (낮을수록 좋음)
      metadata = results['metadatas'][0][i]
      
      # (점수제 로직)
      filter_score = 0
      
      # 점수 1: 카테고리 (가중치 +3)
      if user_filter_dict.get('food_category') == metadata.get('high_level_category'):
        filter_score += 3
        
      # 점수 2: 예산 (가중치 +2)
      if user_filter_dict.get('budget_range') == metadata.get('budget_range'):
        filter_score += 2
        
      # 점수 3: 맵기 (가중치 +2)
      if user_filter_dict.get('spicy_available') == metadata.get('spicy_available'):
        filter_score += 2
        
      # 점수 4: 채식 (가중치 +2)
      if user_filter_dict.get('vegetarian_options') == metadata.get('vegetarian_options'):
        filter_score += 2

      # 점수 5: 분위기/여행 (AND 로직, +1)
      user_suitable_list = user_filter_dict.get('suitable_for', 'N/A').split(',')
      if user_suitable_list != ['N/A']:
        store_str = metadata.get('suitable_for', '')
        if all(req in store_str for req in user_suitable_list):
          filter_score += 1
          
      # 점수 6: 주재료 (OR 로직, +1)
      user_ingr_list = user_filter_dict.get('main_ingredients_list', 'N/A').split(',')
      if user_ingr_list != ['N/A']:
        store_str = metadata.get('main_ingredients_list', '')
        if any(req in store_str for req in user_ingr_list):
          filter_score += 1

      final_results_with_score.append({
        "id": store_id,
        "rag_distance": rag_distance, # Vibe 점수 (낮을수록 좋음)
        "filter_score": filter_score, # Fact 점수 (높을수록 좋음)
        "metadata": metadata
      })
    
    # (★★★ 핵심 수정) 최종 결과를 필터 점수(내림차순), RAG 거리(오름차순)로 정렬
    final_results = sorted(
      final_results_with_score, 
      key=lambda x: (-x['filter_score'], x['rag_distance']), # (1순위: 필터점수, 2순위: RAG거리)
    )[:n_results] # 상위 n_results(5)개만 선택
    
    
    # --- 5. 최종 결과 출력 및 반환 ---
    print("\n--- 3단계: 최종 RAG + 점수제 랭킹 결과 ---")
    if not final_results:
      print("결과를 찾지 못했습니다.")
      return []

    final_recommendation_list = [] # (저장용 리스트)

    # 결과 출력
    for i, item in enumerate(final_results):
      store_name_lookup = df_restaurants.loc[df_restaurants['id'] == int(item['id']), '가게']
      store_name = store_name_lookup.values[0] if not store_name_lookup.empty else 'N/A'
      
      print(f"\n[추천 순위 {i+1}] ID: {item['id']} (가게: {store_name})")
      print(f"  - (Vibe) RAG 점수(거리): {item['rag_distance']:.4f}")
      print(f"  - (Fact) 필터 점수: {item['filter_score']}")
      print(f"  - (Fact) 메타데이터: {item['metadata']}")
      
      # (★★★ 신규) 저장할 딕셔너리 생성
      final_recommendation_list.append({
        "rank": i + 1,
        "restaurant_id": int(item['id']),
        "store_name": store_name,
        "rag_distance": item['rag_distance'],
        "filter_score": item['filter_score']
        # (저장용 CSV에는 용량이 큰 메타데이터는 제외)
      })
      
  except Exception as e:
    print(f"\n[오류] 쿼리 실행 중 오류 발생: {e}")
    return [] # (★★★ 수정) 오류 시 빈 리스트 반환
  
  return final_recommendation_list # (★★★ 수정) 최종 결과 리스트 반환

In [47]:
# --- 6. 헬퍼 함수 (6/7): 사용자 프로필 로드/샘플링 ---
def load_and_sample_user_profiles(filepath, num_to_sample):
  """
  사용자 프로필 CSV를 로드하고, 요청된 개수만큼 샘플링합니다.
  """
  try:
    print(f"\n사용자 프로필 파일({filepath}) 로드 중...")
    user_df_all = pd.read_csv(filepath)
    
    if user_df_all.empty:
      print(f"[오류] 사용자 프로필 파일({filepath})이 비어있습니다.")
      return None
      
    # (샘플링 로직)
    if (
      num_to_sample > 0 and 
      num_to_sample < len(user_df_all)
    ):
      print(f"총 {len(user_df_all)}명의 사용자 프로필 중 {num_to_sample}개를 *랜덤 샘플링*하여 테스트합니다.")
      user_df_to_run = user_df_all.sample(
        n=num_to_sample, 
        random_state=42 # 재현성을 위해 고정
      )
    else:
      print(f"총 {len(user_df_all)}명의 사용자 프로필 *전체*를 테스트합니다.")
      user_df_to_run = user_df_all
      
    return user_df_to_run

  except FileNotFoundError:
    print(f"[오류] 사용자 프로필 파일을 찾지 못했습니다: {filepath}")
    return None
  except Exception as e:
    print(f"[오류] 사용자 프로필 로드 또는 샘플링 중 오류 발생: {e}")
    return None

In [48]:
# --- 7. 헬퍼 함수 (7/7): 메인 파이프라인 실행 ---
def main():
  
  # (시작 시간 기록)
  start_time = time.time()
  
  # 1단계 (가게 DB 로드)
  global df_restaurants
  df_restaurants = load_and_prepare_data(RESTAURANT_DB_FILE) 

  if df_restaurants is None:
    print("가게 DB 로드 실패. 프로그램을 종료합니다.")
    return

  # 2단계 (가게 VectorDB 구축 또는 로드)
  collection = build_vector_db(df_restaurants, clear_db=CLEAR_DB_AND_REBUILD)
  
  if collection is None:
    print("VectorDB 구축 실패. 프로그램을 종료합니다.")
    return
    
  # 3단계 (사용자 프로필 로드 및 검색)
  user_df_to_run = load_and_sample_user_profiles(USER_PROFILE_FILE, NUM_PROFILES_TO_TEST)
  
  if user_df_to_run is None:
    print("사용자 프로필 로드 실패. 프로그램을 종료합니다.")
    return

  print(f"\n--- 총 {len(user_df_to_run)}명의 사용자 프로필로 하이브리드 검색을 시작합니다 ---")
  
  # (★★★ 신규) 최종 결과를 저장할 'Long-Format' 리스트
  all_recommendations_list = []
  
  # (user_df_to_run 순회)
  for i, (original_index, user_profile_row) in enumerate(user_df_to_run.iterrows()):
    print("\n" + "#"*70)
    print(f"### [테스트 {i + 1}/{len(user_df_to_run)}] (Original Index: {original_index}) ###")
    
    # (★★★ 수정) "점수제" 함수 호출
    recommendation_list = run_hybrid_search_with_scoring(
      collection, 
      user_profile_row, 
      df_restaurants
    )
    
    # (★★★ 신규: 사용자 평가 로직 ★★★)
    if recommendation_list: # (결과가 1개 이상일 때)
      n = len(recommendation_list)
      k = min(3, n) # 최소 3개 방문 (결과가 3개 미만이면 n개)
      
      indices = list(range(n))
      random.shuffle(indices) # 추천 순서(rank)와 상관없이 랜덤하게 방문
      
      visited_indices = set(indices[:k])
      
      visited_choices = ['추천', '미추천']
      all_choices = ['미방문', '추천', '미추천']
      
      # (★★★ 수정) recommendation_list의 각 item에 '평가'를 추가
      for j, item_dict in enumerate(recommendation_list):
        if j in visited_indices:
          # (최소 3개 보장)
          item_dict['사용자평가'] = random.choice(visited_choices)
        else:
          # (나머지는 3가지 중 랜덤)
          item_dict['사용자평가'] = random.choice(all_choices)
        
        # (★★★ 수정) 사용자 정보와 평가가 추가된 item_dict를 
        # (★★★) all_recommendations_list에 추가
        item_dict['user_id'] = user_profile_row['user_id']
        item_dict['user_name'] = user_profile_row['name']
        all_recommendations_list.append(item_dict)
    # (★★★ 로직 끝 ★★★)
    
    time.sleep(1) # 로그 확인을 위해 1초 대기
    
  print("\n" + "#"*70)
  print("--- 모든 사용자 프로필 검색이 완료되었습니다. ---")

  # (★★★ 신규) 4단계: 최종 CSV 파일로 저장
  print(f"\n--- 4단계: {len(all_recommendations_list)}개의 추천 결과를 CSV로 저장합니다 ---")
  
  if not all_recommendations_list:
    print("[정보] 저장할 추천 결과가 없습니다.")
  else:
    try:
      reco_df = pd.DataFrame(all_recommendations_list)
      # (컬럼 순서 정리)
      columns_order = [
        'user_id', 'user_name', 'rank', 'restaurant_id', 'store_name', 
        'rag_distance', 'filter_score', '사용자평가'
      ]
      reco_df = reco_df.reindex(columns=columns_order, fill_value=pd.NA)
      
      reco_df.to_csv(RECOMMENDATION_OUTPUT_FILE, index=False, encoding='utf-8-sig')
      print(f"성공! '{RECOMMENDATION_OUTPUT_FILE}' 파일에 저장되었습니다.")
      print(reco_df.head())
      
    except Exception as e:
      print(f"[오류] 최종 추천 결과 CSV 저장 실패: {e}")

  # (종료 시간 계산 및 출력)
  end_time = time.time()
  print(f"\n[작업 완료] 총 실행 시간: {end_time - start_time:.2f} 초")

In [55]:
%%capture captured_output
#%%time
# --- 스크립트 실행 ---
if __name__ == "__main__":
  main()

In [56]:
import datetime

# 1. 타임스탬프 생성 (YYYYMMDD_HHMM)
now = datetime.datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M") # (사용자 요청: YYYYMMDD_24HIMM)

# 2. 파일 이름 정의
stdout_file = f"{timestamp}_stdout.log"
stderr_file = f"{timestamp}_stderr.log"

# 3. STDOUT (일반 로그) 저장
if captured_output.stdout:
    with open(stdout_file, "w", encoding="utf-8") as f:
        f.write(captured_output.stdout)
    print(f"✅ STDOUT이 '{stdout_file}' 파일에 저장되었습니다.")
    
    # 캡처된 STDOUT을 노트북 화면에도 출력 (너무 길 수 있으니 마지막 20줄)
    print("\n--- STDOUT (마지막 20줄) ---")
    print("\n".join(captured_output.stdout.splitlines()[-20:]))
    print("------------------------------")
else:
    print("ℹ️ STDOUT이 비어있습니다.")

# 4. STDERR (오류 로그) 저장
if captured_output.stderr:
    with open(stderr_file, "w", encoding="utf-8") as f:
        f.write(captured_output.stderr)
    print("\n" + "="*50)
    print(f"‼️ [오류 발생] STDERR이 '{stderr_file}' 파일에 저장되었습니다. ‼️")
    print("="*50)
    # 캡처된 STDERR을 노트북 화면에도 출력
    print(captured_output.stderr)
    print("="*50)
else:
    print("✅ STDERR이 비어있습니다 (오류 없음).")

✅ STDOUT이 '20251102_1015_stdout.log' 파일에 저장되었습니다.

--- STDOUT (마지막 20줄) ---
######################################################################
--- 모든 사용자 프로필 검색이 완료되었습니다. ---

--- 4단계: 2500개의 추천 결과를 CSV로 저장합니다 ---
성공! 'recommendation_results_with_ratings.csv' 파일에 저장되었습니다.
     user_id        user_name  rank  restaurant_id   store_name  rag_distance  \
0  user_0446  Giuseppe Romano     1          31371           디토      0.540959   
1  user_0446  Giuseppe Romano     2          29456         최다이닝      0.547199   
2  user_0446  Giuseppe Romano     3          39957  심퍼티쿠시(서울역점)      0.546103   
3  user_0446  Giuseppe Romano     4           8583           가회      0.451722   
4  user_0446  Giuseppe Romano     5           8609           옥정      0.470952   

   filter_score 사용자평가  
0            10   미추천  
1            10   미추천  
2             8   미추천  
3             7    추천  
4             7   미추천  

[작업 완료] 총 실행 시간: 918.58 초
------------------------------
✅ STDERR이 비어있습니다 (오류 없음).
